# Scholaris

> Scholaris is a Python library to set up a research assistant on your local computer, leveraging function calling capabilities. It is designed to help gain insights from scholarly articles in health and life sciences, and is build on top of the [Ollama Python library](https://pypi.org/project/ollama/).

:::{.callout-warning}
This library is under active development and is not yet ready for production use. Report any issues or feature requests on the [GitHub repository](https://github.com/nicomarr/scholaris/issues).
:::

## Installation

**Step 1.** [Download Ollama](https://ollama.com/download) and follow the instructions to install Ollama for your operating system. Then, pull and run [llama3.1](https://ollama.com/library/llama3.1) (parameters: 8B, quantization: Q4_0, size: 4.7 GB) according to the ollama documentation.

**Step 2.** Setup a new virtual environment, such as with Conda. Quick command line instructions on how to install  Miniconda, a free minimal installer for Conda, can be found [here](https://docs.anaconda.com/free/miniconda/#quick-command-line-install).


```sh
$ conda create -n scholaris-env python=3.12
```

**Step 3.** Activate the virtual environment:

```sh
$ conda activate scholaris-env
```

**Step 4.** Install latest scholaris Python package from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/nicomarr/scholaris.git
```


or from [pypi][pypi]:


```sh
$ pip install scholaris
```


[repo]: https://github.com/nicomarr/scholaris
[pypi]: https://pypi.org/project/scholaris/

**Step 5.** Install the following dependencies:

```sh
$ pip install ollama
$ pip install PyPDF2
$ pip install requests
$ pip install tqdm
```

## Getting started

Import the scholaris core module. If you work from a Jupyter notebook environment within a different directory, you may need to add the parent directory of the current working directory to the Python system path as shown below.

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd())) # To add the parent directory to the path
from scholaris.core import *

Make sure the Ollama app is installed on your local computer and Llama 3.1 8B has been downloaded and is running. Then, initialize the `Assistant` class with Llama 3.1 8B, the core functions and default system message. During initialization, messages are printed to indicate whether credentials such as API keys and email are loaded from the environment variables (more on that below), and whether a local data directory already exists or has been created, like so:

In [2]:
assistant = Assistant()

Loaded Semantic Scholar API key from the environment variables.
Loaded email address from the environment variables.
A local directory ../data already exists for storing data files. No of files: 1



:::{.callout-note}
For the examples shown below, a local data directory had already been created in the parent directory prior to initialization, and a PDF file was downloaded and saved to the local data directory for demonstration purposes. You can download the same PDF file, or additional/other files by running the following commands in a Jupyter notebook cell. Alternatively, you can download and add the files manually to the local data directory if the assistant has already been initialized.

    
```python
!mkdir -p ../data
pdf_urls = [
    "https://df6sxcketz7bb.cloudfront.net/manuscripts/144000/144499/jci.insight.144499.v2.pdf",
    # Add more URLs here as needed
]
for url in pdf_urls:
    !curl -o ../data/$(basename {url}) {url}
```
:::

You can also explicitly set the model by passing the model name as an argument to the `Assistant` class. If you do not pass a model name, the default model is ***Llama 3.1*** 8B. An alternative model that supports tool calling and can be run on a standard laptop is Mistral's ***NeMo*** 12 B model. To use the latter model, change the attribute in the `Assistant` class to "mistral-nemo". For more information, read the following [blog post](https://ollama.com/blog/tool-support).

In [ ]:
assistant = Assistant(model="llama3.1:latest")

Loaded Semantic Scholar API key from the environment variables.
Loaded email address from the environment variables.
A local directory ../data already exists for storing data files. No of files: 1



## How to use

To chat with the assistant, simply call the `chat()` method with your prompt as input:

In [ ]:
assistant.chat("Which pdf files do you have access to on the local computer?")



I have access to 1 PDF file on the local computer:

* jci.insight.144499.v2.pdf

By setting `show_progress=True`, you can see the step-by-step progress of the fuction calling. This includes the tool choice, selected arguments, if applicable, and the output of the called function that is being passed back to the LLM to generate the final response.

In [ ]:
assistant.chat("Get the title of this PDF file.", show_progress=True)

Selecting tools...

[{'function': {'name': 'get_titles_and_first_authors', 'arguments': {}}}]
Calling get_titles_and_first_authors() with arguments {}...



Extracting titles and first authors: 100%|██████████| 1/1 [00:07<00:00,  7.69s/it]




Function response:
[
  {
    "title": "Distinct antibody repertoires against endemic human coronaviruses in children and adults",
    "first_author": "Taushif Khan",
    "file_name": "jci.insight.144499.v2.pdf"
  }
]

Generating final response...


The title of the PDF file is:

"Distinct antibody repertoires against endemic human coronaviruses in children and adults"

To get information about a research article from external sources, simply ask the assistant:

In [ ]:
assistant.chat("Get the number of citations for the study entitled 'Tuberculosis and impaired IL-23–dependent IFN-γ immunity in humans homozygous for a common TYK2 missense variant.'")



The study entitled 'Tuberculosis and impaired IL-23–dependent IFN-γ immunity in humans homozygous for a common TYK2 missense variant' has been cited 161 times.

## Conversation history

Show the conversation history by calling the assistant's `show_conversation_history()` method:

In [ ]:
assistant.show_conversion_history()

User: Which pdf files do you have access to on the local computer?

Assistant function calls: get_file_names() with arguments {'ext': 'pdf'}

Function return: List of file names with the specified extensions in the local data directory: jci.insight.144499.v2.pdf

Assistant response: I have access to 1 PDF file on the local computer:

* jci.insight.144499.v2.pdf

User: Get the title of this PDF file.

Assistant function calls: get_titles_and_first_authors() with arguments {}

Function return: [
  {
    "title": "Distinct antibody repertoires against endemic human coronaviruses in children and adults",
    "first_author": "Taushif Khan",
    "file_name": "jci.insight.144499.v2.pdf"
  }
]

Assistant response: The title of the PDF file is:

"Distinct antibody repertoires against endemic human coronaviruses in children and adults"

User: Get the number of citations for the study entitled 'Tuberculosis and impaired IL-23–dependent IFN-γ immunity in humans homozygous for a common TYK2 missens

Clear the conversation history by calling the assistant's `clear_conversation_history()` method:

In [ ]:
assistant.clear_conversion_history()

## Assistant parameters

Show the model by printing the assistant object:

In [ ]:
assistant

Assistant, powered by llama3.1

Or show the model by accessing the assistant's model attribute:

In [ ]:
assistant.model

'llama3.1:latest'

Show the system messages by accessing the assistant's sys_message attribute:

In [ ]:
print(assistant.sys_message)

You are an AI assistant specialized in analyzing research articles.
        Your role is to provide concise, human-readable responses based on information from tools and conversation history.

        Key instructions:
        1. Use provided tools to gather information before answering.
        2. Interpret tool results and provide clear, concise answers in natural language.
        3. If you can't answer with available tools, state this clearly.
        4. Don't provide information if tool content is empty.
        5. Never include raw JSON, tool outputs, or formatting tags in responses.
        6. Format responses as plain text for direct human communication.
        7. Use clear formatting (e.g., numbered or bulleted lists) when appropriate.
        8. Provide article details (e.g., DOI, citation count) in a conversational manner.

        Act as a knowledgeable research assistant, offering clear and helpful information based on available tools and data.
        


## Local data access

By default, the assistant has access to a single directory, called `data`. Within this directory, the assistant can list and read the following file formats and extensions: pdf, txt, md, markdown, csv, and py.
If not already present, the directory is created when the assistant is initialized. If you want to change the directory name, you can do so by passing the desired directory name as an argument to the `Assistant` class when it is initialized. For example, to create a directory called `proprietary_data`, you would initialize the assistant as follows:

In [ ]:
assistant = Assistant(dir_path="../proprietary_data")

Loaded Semantic Scholar API key from the environment variables.
Loaded email address from the environment variables.
Created directory ../proprietary_data for storing data files.



## Tools

By default, the assistant can call a set of core tools or functions which are passed to the `Assistant` as a dictionary when it is initialized. With these tools or functions, the assistant will be able to get a list of file names in a specific data directory, can extract content from these files, and summarize them. In addition, the assistant can make API calls to external data sources, such as [OpenAlex](https://openalex.org) or [Semantic Scholar](https://www.semanticscholar.org), to retrieve information about a large number of scholarly articles. The tools available to the assistant can be viewed by accessing the assistant's `list_tools()` method as follows:

In [ ]:
assistant.list_tools()

get_file_names
extract_text_from_pdf
get_titles_and_first_authors
summarize_local_document
describe_python_code
query_openalex_api
query_semantic_scholar_api
respond_to_generic_queries


:::{.callout-tip}
You can learn more details about the core tools by visiting the Source Code page, which lists each function and provides a brief description of its purpose, functionality, required arguments, and usage (the docstring). This information helps you understand the available tools and how the LLM uses them. Alternatively, if you prefer to see the schema for all tools the assistant can call, you can execute the `assistant.get_tools_schema()` method.
:::

## Authentication for tool calling and API access (optional)

Some tools take optional authentication parameters, such as an API key or email. For example, the `query_semantic_scholar` tool takes an optional API key to access the Semantic Scholar API, which will increase the API rate limit. Request a Semantic Scholar API Key [here](https://www.semanticscholar.org/product/api/tutorial). Similarly, the `query_openaplex_api` tool takes an optional email parameter to access the OpenAlex API, which is recommended as a best practice and kindly requested by the [API provider](https://docs.openalex.org/how-to-use-the-api/rate-limits-and-authentication#the-polite-pool). 

The best way to pass these parameters is to set them as environment variables, with the following key names: `SEMANTIC_SCHOLAR_API_KEY` and `EMAIL`. The Assistant class will automatically read these environment variables when initialized and pass them to the tools that require them. Alternatively, you can pass the Semantic Scholar API key and your email by simply adding the authentication argument when initializing the Assistant class, as shown below:

In [ ]:
authentication = {
    "SEMANTIC_SCHOLAR_API_KEY": "your_api_key",
    "EMAIL": "your_email@example.com"
}
assistant = Assistant(authentication=authentication)

A local directory ../data already exists for storing data files. No of files: 1



 If you want to change the core functions, you can do so by passing the desired core functions as an argument to the Assistant class when it is initialized. For example, to limit the assitant's ability to respond to generic questions and to access external data by making requests to the OpenAlex and Semantic Scholar API's, you would initialize the assistant as follows:

In [ ]:
assistant = Assistant(tools = {
    "query_openalex_api": query_openalex_api,
    "query_semantic_scholar_api": query_semantic_scholar_api,
    "respond_to_generic_queries": respond_to_generic_queries,
    })

Loaded Semantic Scholar API key from the environment variables.
Loaded email address from the environment variables.
A local directory ../data already exists for storing data files. No of files: 1



:::{.callout-note}
The research assistent is set up so that it has to use a tool to generate a final response to a user's prompt. This is to ensure that the assistant is primarily providing information which is relevant for health and life sciences. Otherwise it will abort the conversation, like so:
:::

In [ ]:
assistant = Assistant(tools = {})
assistant.chat("What is the capital of France?")

Loaded Semantic Scholar API key from the environment variables.
Loaded email address from the environment variables.
A local directory ../data already exists for storing data files. No of files: 1

No tools provided! Please add tools to the assistant.
No tool calls found in the response. Adding an empty tool_calls list to the conversation history. Aborting...



## Developer Guide

### Defining new tools

You can define new functions to be used by the assistant as tools. To simplify this process, a decorator called `@json_schema_decorator` is provided so it is not necessary to define the schema for the function. The schema is automatically generated based on the function's annotation and docstring. 

::: {.callout-tip}
Here are key points to consider when defining a new tool:

- Use type hints in the function signature to define the input and output types.
- Use docstrings, as shown below, to describe the function's purpose and the expected input and output.
- Use the `@json_schema_decorator` decorator to automatically generate the schema for the function.
- Ensure the output is a string (such as a JSON-formatted string) that can be passed back to the LLM to generate the final response.
:::

:::{.callout-important}
It's crucial to understand that this metadata (function name, type hints, and docstring) is all the information the LLM has access to when deciding which function to call and how to use it. The LLM does not have access to or information about the actual source code or implementation of the functions (unless explicitly provided). Therefore, the metadata must be comprehensive and accurate to ensure proper function selection and usage by the LLM. 

:::

The following example shows how to define a new tool called `multiply_two_integers` that takes two integers as input and returns a string as output:

In [ ]:
@json_schema_decorator
def multiply_two_integers(a: int, b: int) -> int:
    """
    A function to multiply two numbers.

    Args:
        a (int): First integer.
        b (int): Second integer.

    Returns:
        str: The product of the two numbers, as a string.
    """

    if not isinstance(a, int) or not isinstance(b, int):
        return "Error: Inputs must be integers."
    
    return str(a * b)

To ensure the JSON schema is generated correctly, you can call the `json_schema` attribute of the function:

In [ ]:
multiply_two_integers.json_schema

{'type': 'function',
 'function': {'name': 'multiply_two_integers',
  'description': 'A function to multiply two numbers.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': 'First integer.'},
    'b': {'type': 'integer', 'description': 'Second integer.'}},
   'required': ['a', 'b']}}}

Alternatively, you can use the `generate_json_schema` function:

In [ ]:
generate_json_schema(multiply_two_integers)

{'type': 'function',
 'function': {'name': 'multiply_two_integers',
  'description': 'A function to multiply two numbers.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': 'First integer.'},
    'b': {'type': 'integer', 'description': 'Second integer.'}},
   'required': ['a', 'b']}}}

### Adding tools

You can add new tools by passing a dictionary of new tools to the `Assistant` class when it is initialized. Use the `add_tools` argument to add new tools to the assistant. This will merge the new tools with the existing tools. For example, to add the new tool called `multiply_two_integers` to the assistant, you would initialize the assistant as follows:

In [ ]:
assistant = Assistant(add_tools = {"multiply_two_integers": multiply_two_integers})

Loaded Semantic Scholar API key from the environment variables.
Loaded email address from the environment variables.
A local directory ../data already exists for storing data files. No of files: 1



You can confirm that the new tool has been added to the list of existing tools by using the `list_tools()` method:

In [ ]:
assistant.list_tools()

get_file_names
extract_text_from_pdf
get_titles_and_first_authors
summarize_local_document
describe_python_code
query_openalex_api
query_semantic_scholar_api
respond_to_generic_queries
multiply_two_integers


In [ ]:
assistant.chat("What is the product of 4173 and 351?", show_progress=True)

Selecting tools...

[{'function': {'name': 'multiply_two_integers', 'arguments': {'a': 4173, 'b': 351}}}]
Calling multiply_two_integers() with arguments {'a': 4173, 'b': 351}...

Function response:
1464723

Generating final response...


The product of 4173 and 351 is 1,464,723.

### Adding methods

You can add new methods to the `Assistant` class by using the `add_to_class()` decorator function. For example, the `clear_conversion_history` method was added to the `Assistant` class as follows:

In [ ]:
@add_to_class(Assistant)
def clear_conversion_history(self):
    """Clear the conversation history."""
    self.messages = [{'role': "system", 'content': self.sys_message},]